# Import dependencies

In [ ]:
!pip install scikit-opt

In [ ]:
import glob
import random
from tqdm import tqdm
from attr import dataclass

import numpy as np
import pandas as pd
from sko.PSO import PSO
from sklearn.impute import SimpleImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/MyDrive/collected_metrics.zip collected_metrics.zip
!unzip collected_metrics
!rm collected_metrics.zip

Archive:  collected_metrics.zip
   creating: collected_metrics/
  inflating: __MACOSX/._collected_metrics  
   creating: collected_metrics/Jailer/
  inflating: __MACOSX/collected_metrics/._Jailer  
   creating: collected_metrics/Paper/
  inflating: __MACOSX/collected_metrics/._Paper  
   creating: collected_metrics/ClickHouse-Native-JDBC/
  inflating: __MACOSX/collected_metrics/._ClickHouse-Native-JDBC  
   creating: collected_metrics/neo4j/
  inflating: __MACOSX/collected_metrics/._neo4j  
   creating: collected_metrics/doris-spark-connector/
  inflating: __MACOSX/collected_metrics/._doris-spark-connector  
   creating: collected_metrics/spring-data/
  inflating: __MACOSX/collected_metrics/._spring-data  
   creating: collected_metrics/restheart/
  inflating: __MACOSX/collected_metrics/._restheart  
   creating: collected_metrics/jdbi/
  inflating: __MACOSX/collected_metrics/._jdbi  
   creating: collected_metrics/jeddict/
  inflating: __MACOSX/collected_metrics/._jeddict  
   creatin

# Declare a types

In [ ]:
@dataclass
class Gene:
    metric: str

In [ ]:
@dataclass
class Chromosome:
    genes: list[Gene]

In [ ]:
@dataclass
class Population:
    chromosomes: list[Chromosome]

# Load data

In [ ]:
type(pd.read_csv('collected_metrics/Jailer/class.csv'))

pandas.core.frame.DataFrame

In [ ]:
def load_dataframe(filename, columns):
    try:
        return pd.read_csv(filename)._get_numeric_data().head(100).astype(float)
    except pd.errors.EmptyDataError:
        df = pd.DataFrame(columns=columns, dtype='float64')
        df.loc[0] = [np.nan] * len(columns)
        return df
    except FileNotFoundError:
        return None

## Get all paths

In [ ]:
class_metrics_csvs = glob.glob("collected_metrics/*/class.csv")
method_metrics_csvs = glob.glob("collected_metrics/*/method.csv")

## Get class and methods metrics names

In [ ]:
class_columns: list[str] = list(pd.read_csv(class_metrics_csvs[0]).columns)[3:]
method_columns: list[str] = list(pd.read_csv(method_metrics_csvs[0]).columns)[3:]

## Instantiate a repository class

In [ ]:
def get_full_path_class(repository_name: str) -> str:
    return f"collected_metrics/{repository_name}/class.csv"


def get_full_path_method(repository_name: str) -> str:
    return f"collected_metrics/{repository_name}/method.csv"

In [ ]:
class Repo:
    def __init__(self, name: str):
        self.name: str = name
        self.is_load: bool = False

    def load(self):
        if self.is_load:
            return self

        class_dataframe_path: str = get_full_path_class(self.name)
        method_dataframe_path: str = get_full_path_method(self.name)

        # TODO: add to __init__
        self._class_df = load_dataframe(class_dataframe_path, class_columns)
        self._method_df = load_dataframe(method_dataframe_path, class_columns)

        self.is_load = True

    def get_class_df(self):
        if not self.is_load:
            self.load()
        return self._class_df

    def get_method_df(self):
        if not self.is_load:
            self.load()
        return self._method_df

## Impute all data

In [ ]:
class ImputedRepo(Repo):
    def __init__(self, name: str):
        super().__init__(name)
        self.strategy: str = "mean"

    def _impute_df(self, df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
        miss_mean_imputer = SimpleImputer(missing_values=np.nan, strategy=self.strategy)
        miss_mean_imputer = miss_mean_imputer.fit(df)
        imputed_df = miss_mean_imputer.transform(df)

        return pd.DataFrame(imputed_df, columns=columns)

    def get_class_df(self):
        return self._impute_df(super().get_class_df(), class_columns)

    def get_method_df(self):
        return self._impute_df(super().get_method_df(), method_columns)

## Cache all repos

In [ ]:
class CachedRepo(ImputedRepo):
    def __init__(self, name: str):
        super().__init__(name)

        self._class_df = None
        self._method_df = None

    def get_class_df(self):
        if self._class_df is None:
            self._class_df = super().get_class_df()
        return self._class_df

    def get_method_df(self):
        if self._method_df is None:
            self._method_df = super().get_method_df()
        return self._method_df

## Collecting repositories data

In [ ]:
import os

repositories_names: list[str] = list(os.walk("collected_metrics"))[0][1]

repositories: list[CachedRepo] = list(map(
    lambda repository_name: CachedRepo(repository_name),
    repositories_names
))

# Error functionality

In [ ]:
class ErrorRepositoryResults:
    def __init__(self, repo: CachedRepo, error, distance):
        self.repo = repo
        self.error = error
        self.distance = distance

    def class_error(self, columns):
        class_df = self.repo.get_class_df()

        try:
            return self.error(class_df, class_df[columns], self.distance)
        except KeyError:
            raise ValueError()

    def method_error(self, columns):
        method_df = self.repo.get_method_df()

        try:
            return self.error(method_df, method_df[columns], self.distance)
        except KeyError:
            raise ValueError()

## Error function

In [ ]:
import tensorflow as tf


def kruskal_stress(n_df, l_df, distance_tf):
    n_df_tensor = tf.convert_to_tensor(n_df.values, dtype=tf.float64)
    l_df_tensor = tf.convert_to_tensor(l_df.values, dtype=tf.float64)

    n_distances = distance_tf(n_df_tensor[:, None, :], n_df_tensor[None, :, :])
    l_distances = distance_tf(l_df_tensor[:, None, :], l_df_tensor[None, :, :])

    numerator = tf.reduce_sum(tf.square(n_distances - l_distances))
    denominator = tf.reduce_sum(tf.square(n_distances))

    stress = tf.sqrt(numerator) / tf.sqrt(denominator)

    return stress.numpy()

In [ ]:
def eq_distance(vec1, vec2):
    vec1 = tf.convert_to_tensor(vec1)
    vec2 = tf.convert_to_tensor(vec2)

    squared_diff = tf.reduce_sum(tf.square(vec1 - vec2), axis=2)

    return tf.sqrt(squared_diff)

# Running experiments

In [ ]:
from abc import ABC, abstractmethod
from typing import Any


class OptimizationAlgorithmInterface(ABC):
    @abstractmethod
    def run_experiments(self):
        pass

    def aggregate_results(self, results: np.ndarray) -> Any:
        return results.mean()

In [ ]:
from typing import Final


class GenericAlgorithm(OptimizationAlgorithmInterface):
    def __init__(
            self,
            metrics_labels: list[str],
            offsprings_numbers=10,
            population_size=10,
            mutation_rate=0.5,
            max_iterations=100,
            max_repeats=5
        ):
        self.population_size: Final[int] = population_size
        self.metrics_labels: Final[list[str]] = metrics_labels
        self.offsprings_numbers = offsprings_numbers
        self.mutation_rate = mutation_rate
        self.max_iterations = max_iterations
        self.max_repeats = max_repeats

    def _get_initial_population(self, sample_size: int) -> list[list[str]]:
        # TODO: type
        population: list[list[str]] = []

        for _ in range(self.population_size):
            while True:
                chromosome: Final[list[str]] = random.sample(self.metrics_labels, sample_size)

                # test that chromosome contains unique genes
                assert sorted(chromosome) == sorted(list(set(chromosome)))

                if chromosome in population:
                    continue

                population.append(chromosome)
                break

        return population

    def _get_parents(self, population: list[list[str]]) -> tuple[list[list[str]], list[list[str]]]:
        mothers = population[:(2 * self.offsprings_numbers):2]
        fathers = population[1:(2 * self.offsprings_numbers + 1):2]

        return mothers, fathers

    def _crossover(self, mother: list[str], father: list[str]) -> list[str]:
        assert len(mother) == len(father)

        possible_offsprings = [mother, father]

        for border_left in range(0, len(mother) + 1):
            for border_right in range(border_left, len(mother) + 1):
                offspring_1: Final[list[str]] = mother[:border_left] + \
                    father[border_left:border_right] + mother[border_right:]

                if len(set(offspring_1)) == len(offspring_1):
                    possible_offsprings.append(offspring_1)

                offspring_2: Final[list[str]] = father[:border_left] + \
                    mother[border_left:border_right] + father[border_right:]

                if len(set(offspring_2)) == len(offspring_2):
                    possible_offsprings.append(offspring_2)

        offspring: Final[list[str]] = random.choice(possible_offsprings)

        assert len(set(offspring)) == len(offspring)

        return offspring

    def _mutate(self, chromosome: list[str]) -> list[str]:
        new_chromosome = chromosome[:]

        for i in range(len(new_chromosome)):
            if random.random() < self.mutation_rate:
                while True:
                    random_gene = random.choice(class_columns)
                    if random_gene not in new_chromosome:
                        new_chromosome[i] = random_gene
                        break

        assert len(set(new_chromosome)) == len(new_chromosome)

        return new_chromosome

    def _get_class_fitness(self, columns: list[str]) -> np.float64:
        results = []

        for repository in repositories:
            try:
                result = ErrorRepositoryResults(repository, kruskal_stress, eq_distance).class_error(columns)
            except ValueError:
                continue
            results.append(result)

        return self.aggregate_results(np.array(results))

    def _get_method_fitness(self, columns: list[str]) -> np.float64:
        results = []

        for repository in repositories:
            result = ErrorRepositoryResults(repository, kruskal_stress, eq_distance).method_error(columns)
            results.append(result)

        return self.aggregate_results(np.array(results))

    # def _get_class_fitness(self, repository) -> np.float64:
    #     return self._get_total_fitness(ErrorRepositoryResults(repository, kruskal_stress, eq_distance).class_error, class_columns)

    # def _get_method_fitness(self, repository) -> np.float64:
    #     return self._get_total_fitness(ErrorRepositoryResults(repository, kruskal_stress, eq_distance).method_error, method_columns)

    def _get_population_fitness(self, population: list[list[str]], fitness_function):
        fitness_results = []
        for chromosome in population:
            fitness_value = fitness_function(chromosome)
            fitness_results.append((chromosome, fitness_value))
        fitness_results.sort(key=lambda x: x[1])
        return fitness_results

    def _replace_population(
            self,
            population: list[list[str]],
            new_individuals: list[list[str]],
            fitness_function
    ):
        # Add new chromosomes
        population.extend(new_individuals)

        # Sort them by fitness score
        fitness_results = self._get_population_fitness(population, fitness_function)

        population = [a[0] for a in fitness_results]
        # Remove the chromosomes that have low fitness score
        population = population[:self.population_size]

        # Return the updated population
        return population

    def _evolution_step(self, population: list[list[str]], fitness_function):
        # Get the fittest n_offsprings parents from the given population
        mothers, fathers = self._get_parents(population)

        mothers: list[list[str]]
        fathers: list[list[str]]

        # Here we will store the offsprings
        offsprings: list[list[str]] = []

        # Iterate each pair of mother and father
        for mother, father in zip(mothers, fathers):
            mother: Final[list[str]]
            father: Final[list[str]]

            # Make a crossover of them, mutate an offspring
            offspring: Final[list[str]] = self._mutate(self._crossover(mother, father))

            # Add the offspring to the list of offsprings
            offsprings.append(offspring)

        # Update the population with new offsprings
        new_population = self._replace_population(population, offsprings, fitness_function)

        # Return the updated population
        return new_population

    def _run_experiments_on_n_metrics(self, metrics_amount: int, fitness_function):
        # TODO: type
        population: list[list[str]] = self._get_initial_population(metrics_amount)

        prev_value = -1
        repeats = 0

        for iteration in range(self.max_iterations):
            population = self._evolution_step(population, fitness_function)

            fitness_scores = self._get_population_fitness(population, fitness_function)
            best_fitness = fitness_scores[0][1]
            best_result = fitness_scores[0][0]

            if prev_value != best_fitness:
                prev_value = best_fitness
                repeats = 1
            else:
                repeats += 1

            if repeats == self.max_repeats:
                break

            print(f"Best fitness: {best_fitness}")
            print(f"Best result: {best_result}")
            print(f"Repeats: {repeats}")
            print(f"Previous value: {prev_value}")
            print()

        fitness_scores = self._get_population_fitness(population, fitness_function)

        print(f"subset size: {metrics_amount}")
        print(f"Final best fitness: {fitness_scores[0][1]}")
        print(f"Final best result: {fitness_scores[0][0]}")
        print("-------------")
        print()

    def run_experiments(self):
        for metrics_amount in range(1, len(self.metrics_labels)):
            self._run_experiments_on_n_metrics(metrics_amount, self._get_class_fitness)

        for metrics_amount in range(1, len(self.metrics_labels)):
            self._run_experiments_on_n_metrics(metrics_amount, self._get_method_fitness)

In [ ]:
class ExperimentRunner:
    # TODO: types
    def __init__(self, repositories, optimization_algorithm: OptimizationAlgorithmInterface) -> None:
        self.repositories = repositories
        self.optimization_algorithm: OptimizationAlgorithmInterface = optimization_algorithm

    def run_experiments(self):
        self.optimization_algorithm.run_experiments()

In [ ]:
generic_algorithm_runner = ExperimentRunner(repositories, GenericAlgorithm(class_columns))

In [ ]:
generic_algorithm_runner.run_experiments()

Fitness score: [(['stringLiteralsQty'], 0.9530206974242162), (['totalMethodsQty'], 0.9781169946682959), (['fanout'], 0.9854170487299363), (['totalFieldsQty'], 0.9924250647195014), (['mathOperationsQty'], 0.9927136794218588), (['finalFieldsQty'], 0.9956928594269098), (['finalFieldsQty'], 0.9956928594269098), (['staticMethodsQty'], 0.9970890396103446), (['maxNestedBlocksQty'], 0.9976636170409301), (['maxNestedBlocksQty'], 0.9976636170409301), (['publicFieldsQty'], 0.9984945975970464), (['abstractMethodsQty'], 0.999443438930206), (['synchronizedMethodsQty'], 0.9997576459033131), (['synchronizedMethodsQty'], 0.9997576459033131), (['synchronizedFieldsQty'], 1.0)]
Best fitness: 0.9530206974242162
Best result: ['stringLiteralsQty']
Repeats: 1
Previous value: 0.9530206974242162

Fitness score: [(['stringLiteralsQty'], 0.9530206974242162), (['totalMethodsQty'], 0.9781169946682959), (['fanout'], 0.9854170487299363), (['fanout'], 0.9854170487299363), (['totalFieldsQty'], 0.9924250647195014), (['m

KeyboardInterrupt: 

In [ ]:
class PSOAlgorithm(OptimizationAlgorithmInterface):
    def __init__(self, repos_train):
        self.n = 10
        self.repos_train = repos_train

    def _all_nans(self, dataset):
        flag = True

        for index, row in dataset.iterrows():
            for cell in row:
                if not pd.isnull(cell):
                    flag = False
                    break
            if not flag:
                break

        return flag

    def _total_error_method(self, columns):
        results = []

        # collect errors
        for repo in repositories:
            try:
                if self._all_nans(repo.get_method_df()):
                    continue
            except ValueError:
                continue

            try:
                result = ErrorRepositoryResults(repo, kruskal_stress, eq_distance).method_error(columns)
            except ValueError:
                continue

            results.append(result)

        return self.aggregate_results(np.array(results))

    def _total_error_class(self, columns):
        results = []

        # collect errors
        for repo in repositories:
            try:
                if self._all_nans(repo.get_class_df()):
                    continue
            except ValueError:
                continue

            try:
                result = ErrorRepositoryResults(repo, kruskal_stress, eq_distance).class_error(columns)
                # print("result: " + str(result))
            except ValueError:
                continue

            results.append(result)

        return self.aggregate_results(np.array(results))

    def _class_error_wrapper(self, repo):
        def error(x):
            x = np.array(x)

            subset = [class_columns[i] for i in x.argsort()[-self.n:][::-1]]

            if len(subset) == 0:
                return float('inf')  # Avoid empty subset
            else:
                try:
                    return ErrorRepositoryResults(repo, kruskal_stress, eq_distance).class_error(subset)
                except ValueError:
                    return float('inf')

        return error

    def _method_error_wrapper(self, repo):
        def error(x):
            x = np.array(x)

            subset = [class_columns[i] for i in x.argsort()[-self.n:][::-1]]

            if len(subset) == 0:
                return float('inf')  # Avoid empty subset
            else:
                try:
                    return ErrorRepositoryResults(repo, kruskal_stress, eq_distance).method_error(subset)
                except ValueError:
                    return float('inf')

        return error

    def _run_experiments_on_classes(self):
        lower_bounds = [0] * len(class_columns)
        upper_bounds = [1] * len(class_columns)

        for subset_size in range (1, len(class_columns) + 1):
            best_subset = []
            best_error = 1
            for i in tqdm(range(len(self.repos_train))):
                if self._all_nans(self.repos_train[i].get_class_df()):
                    continue

                repo = self.repos_train[i]

                N = subset_size

                try:
                    pso = PSO(func=self._class_error_wrapper(repo), n_dim=len(class_columns),pop=30, max_iter=30, lb=lower_bounds, ub=upper_bounds,w=0.6, c1=2, c2=2, verbose=False)
                    pso.record_mode = True
                    pso.run()
                except ValueError:
                    continue

                x = np.array(pso.gbest_x)[0]

                optimal_subset = [class_columns[i] for i in x.argsort()[-N:][::-1]]
                optimal_subset_error = self._total_error_class(optimal_subset)

                print("optimal subset: " + str(optimal_subset))
                print("optimal subset error: " + str(optimal_subset_error))

                if optimal_subset_error < best_error:
                    best_error = optimal_subset_error
                    best_subset = optimal_subset

            print(f"subset: {best_subset}")
            print(f"subset size: {len(best_subset)}")
            print(f"error: {best_error}")

    def _run_experiments_on_methods(self):
        lower_bounds = [0] * len(method_columns)
        upper_bounds = [1] * len(method_columns)

        for subset_size in range (1, len(method_columns) + 1):
            best_subset = []
            best_error = 1
            for i in tqdm(range(len(self.repos_train))):
                if self._all_nans(self.repos_train[i].get_method_df()):
                    continue

                repo = self.repos_train[i]

                N = subset_size

                try:
                    pso = PSO(func=self._method_error_wrapper(repo), n_dim=len(method_columns),pop=30, max_iter=30, lb=lower_bounds, ub=upper_bounds,w=0.6, c1=2, c2=2, verbose=False)
                    pso.record_mode = True
                    pso.run()
                except ValueError:
                    continue

                x = np.array(pso.gbest_x)[0]

                optimal_subset = [method_columns[i] for i in x.argsort()[-N:][::-1]]
                optimal_subset_error = self._total_error_method(optimal_subset)

                print("optimal subset: " + str(optimal_subset))
                print("optimal subset error: " + str(optimal_subset_error))

                if optimal_subset_error < best_error:
                    best_error = optimal_subset_error
                    best_subset = optimal_subset

            print(f"subset: {best_subset}")
            print(f"subset size: {len(best_subset)}")
            print(f"error: {best_error}")

    def run_experiments(self):
        self._run_experiments_on_classes()

        self._run_experiments_on_methods()

In [ ]:
from sklearn.model_selection import train_test_split
repos_train, repos_test = train_test_split(repositories, test_size=0.2, random_state=1)

In [ ]:
pso_algorithm_runner = ExperimentRunner(repositories, PSOAlgorithm(repos_train))

In [ ]:
pso_algorithm_runner.run_experiments()

  1%|          | 1/96 [00:02<03:40,  2.32s/it]

optimal subset: ['assignmentsQty']
optimal subset error: 0.9931091822373502


  2%|▏         | 2/96 [00:04<03:21,  2.15s/it]

optimal subset: ['lambdasQty']
optimal subset error: 0.9996979617848296


  3%|▎         | 3/96 [00:05<02:45,  1.78s/it]

...

optimal subset: ['rfc', 'mathOperationsQty', 'fanin', 'cboModified', 'returnsQty']
optimal subset error: 0.9784050247049525


 23%|██▎       | 22/96 [00:17<01:03,  1.16it/s]

optimal subset: ['constructor', 'variablesQty', 'comparisonsQty', 'assignmentsQty', 'lambdasQty']
optimal subset error: 0.9903451056529955


 23%|██▎       | 22/96 [00:18<01:02,  1.18it/s]


KeyboardInterrupt: 